In [2]:
import pandas as pd
import nltk
import spacy
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from symspellpy import SymSpell, Verbosity


In [3]:
df = pd.read_csv(r"D:\epita class notes\semester - 3\action learnign\project repository\Hate_speech_detection_using_data_augmentation\Hate_speech_detection_using_data_augmentation\data\labeled_data.csv")
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [4]:
df_filtered = df.head(1000)
df_filtered

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
995,1017,3,0,3,0,1,&#128514;&#128514;&#128514;&#128514; RT @SMASH...
996,1018,3,0,3,0,1,&#128514;&#128514;&#128514;&#128514; bitch if ...
997,1019,3,0,2,1,1,&#128514;&#128514;&#128514;&#128514; these fol...
998,1020,6,0,6,0,1,&#128514;&#128514;&#128514;&#128514;&#128514; ...


In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\edwin
[nltk_data]     victor\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [6]:
nlp = spacy.load("en_core_web_sm") #inorder to load this model we have to download this model from terminal

In [7]:
def clean_text(text):
    text = text.lower() #converting the tweets to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text) #removing url's
    text = re.sub(r'@\w+', '', text)  # removing the username from the tweet
    text = re.sub(r'[^a-zA-Z\s]', '', text) #removing non alpha-numeric character
    text = re.sub(r'\s+', ' ', text) #removing extra space
    text = re.sub(r'\brt\b', '', text) #removing rt from retweeted tweets
    text = re.sub(r'\b[a-zA-Z]*\*+[a-zA-Z]*\b', '[censored]', text) #replacing obscene words with censored
    text = text.strip()
    return text
    

In [15]:
def clean_dataset(df, sym_spell):
    df['cleaned_tweet'] = df['tweet'].apply(clean_text)
    df['corrected_tweet'] = df['cleaned_tweet'].apply(lambda x: correct_spelling(x, sym_spell))
    return df[['corrected_tweet']]

In [9]:
cleaned_df_1 = clean_dataset(df_filtered)
cleaned_df_1

C:\Users\edwin victor\AppData\Local\Temp\ipykernel_13548\860863289.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweet'] = df['tweet'].apply(clean_text)


,cleaned_tweet
0,as a woman you shouldnt complain about cleanin...
1,boy dats coldtyga dwn bad for cuffin dat hoe i...
2,dawg you ever fuck a bitch and she start to c...
3,she look like a tranny
4,the shit you hear about me might be true or it...
...,...
995,murda out here sucking bitches howdhow
996,bitch if youre the hobbit you need to let me k...
997,these folks so bad im just here to talk trash
998,brittany bitch u my dog man


In [10]:
def initialize_symspell():
    sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
    dictionary_path = "frequency_dictionary_en_82_765.txt"
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
    return sym_spell

In [11]:
def correct_spelling(text, sym_spell):
    corrected_words = []
    for word in text.split():
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
        if suggestions:
            corrected_words.append(suggestions[0].term)  
        else:
            corrected_words.append(word) 
    return ' '.join(corrected_words)

In [16]:
sym_spell = initialize_symspell()

2024-12-28 20:26:37,044: E symspellpy.symspellpy] Dictionary file not found at frequency_dictionary_en_82_765.txt.


In [17]:
cleaned_df_1 = clean_dataset(df_filtered, sym_spell)
print(cleaned_df_1)

                                       corrected_tweet
0    as a woman you shouldnt complain about cleanin...
1    boy dats coldtyga dwn bad for cuffin dat hoe i...
2    dawg you ever fuck a bitch and she start to cr...
3                               she look like a tranny
4    the shit you hear about me might be true or it...
..                                                 ...
995             murda out here sucking bitches howdhow
996  bitch if youre the hobbit you need to let me k...
997      these folks so bad im just here to talk trash
998                        brittany bitch u my dog man
999                           smh bitch sellin her ass

[1000 rows x 1 columns]


C:\Users\edwin victor\AppData\Local\Temp\ipykernel_13548\1621942341.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_tweet'] = df['tweet'].apply(clean_text)
C:\Users\edwin victor\AppData\Local\Temp\ipykernel_13548\1621942341.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['corrected_tweet'] = df['cleaned_tweet'].apply(lambda x: correct_spelling(x, sym_spell))
